In [1]:
import cv2
import numpy as np
import gradio as gr

def detect_traffic_sign(image):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # ---------- COLOR RANGES ----------
    red1 = cv2.inRange(hsv, (0,70,50), (10,255,255))
    red2 = cv2.inRange(hsv, (170,70,50), (180,255,255))
    red_mask = red1 + red2

    yellow_mask = cv2.inRange(hsv, (20,100,100), (35,255,255))
    green_mask = cv2.inRange(hsv, (36,50,50), (89,255,255))
    blue_mask = cv2.inRange(hsv, (90,50,50), (130,255,255))

    # ---------- PIXEL COUNT ----------
    r, y, g, b = map(np.sum, [red_mask, yellow_mask, green_mask, blue_mask])

    # ---------- SHAPE DETECTION ----------
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)
        area = cv2.contourArea(cnt)

        if area < 1000:
            continue

        # STOP sign (octagon)
        if len(approx) == 8 and r > 50000:
            return "🛑 STOP SIGN"

        # NO ENTRY (circle + red)
        if len(approx) > 8 and r > 60000:
            return "⛔ NO ENTRY SIGN"

        # SPEED LIMIT (circle)
        if len(approx) > 8 and r > 30000:
            return "🚫 SPEED LIMIT SIGN"

    # ---------- TRAFFIC SIGNAL ----------
    if r > y and r > g:
        return "🔴 RED SIGNAL – STOP"
    elif y > r and y > g:
        return "🟡 YELLOW SIGNAL – READY"
    elif g > r and g > y:
        return "🟢 GREEN SIGNAL – GO"

    # ---------- PEDESTRIAN ----------
    if b > 40000:
        return "🚸 PEDESTRIAN CROSSING"

    return "⚠️ Traffic Sign Not Detected"


# ---------- GRADIO UI ----------
interface = gr.Interface(
    fn=detect_traffic_sign,
    inputs=gr.Image(type="numpy", label="Upload Traffic Sign Image"),
    outputs=gr.Textbox(label="Detected Traffic Sign"),
    title="🚦 Multi Traffic Sign Detection System",
    description="Detect Traffic Signal, Stop, No Entry, Pedestrian & Speed Limit signs"
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
